In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [4]:
df = pd.read_csv("CSV_Data_For_3.csv")

In [3]:
df.head()

,UserID,Order,Time
0,200000000.0,"[0,1,0]","[0,0,1]"
1,200000000.0,"[0,1,0]","[0,0,1]"
2,200000000.0,"[1,0,0]","[0,1,0]"
3,200000000.0,"[1,0,0]","[0,0,1]"
4,200000000.0,"[1,0,0]","[0,0,1]"


In [5]:
numpy_data = df.to_numpy()

In [6]:
sample_data = numpy_data[0:-1]
sample_data

array([[200000000.0, '[0,1,0]', '[0,0,1]'],
       [200000000.0, '[0,1,0]', '[0,0,1]'],
       [200000000.0, '[1,0,0]', '[0,1,0]'],
       ...,
       [5670000000000000.0, '[1,0,0]', '[1,0,0]'],
       [5690000000000000.0, '[1,0,0]', '[0,0,1]'],
       [5820000000000000.0, '[1,0,0]', '[0,0,1]']], dtype=object)

In [7]:
for val in sample_data:
    val[1] = np.array(eval(val[1]))
    val[2] = np.array(eval(val[2]))
    #print(val[2])

In [8]:
 # order_food_dict = {uniq: [[order, time], [order, time],   [order, time], [order, time]],   uniq2: [[order, time], [order, time],   [order, time], [order, time]]}

order_food_dict = {}
for sample in sample_data:
    #print(f"Key - {sample[0]}     Value - {' '.join([a for a in sample[1].split()[0:-1]])} {sample[2]}")
    #print("\n")
    key = sample[0]
    values = sample[1:]
    # check if the key is in the dictionary
    if order_food_dict.get(key) is not None:
         #   - get the value for that key
        existing_values = order_food_dict.get(key)
        # append new value to existing values
        existing_values.append(values)
        order_food_dict[key] = existing_values
        pass
    else:
        # create new values array
        new_values = []
        new_values.append(values)
        order_food_dict[key] = new_values
    
   
    pass

In [9]:
WINDOW_SIZE = 1
ORDER_SIZE = 3


new_dummy_data = []

for (k, v) in order_food_dict.items():
    customer_orders = order_food_dict.get(k)
    dummy_data = np.asarray(customer_orders)
    for j in range(len(dummy_data)):
        curr = list(dummy_data[j])
        if j + 1 < len(dummy_data):
            curr.append(dummy_data[j+1][1])
            pass
        else:
            curr.append([0,0,0])

        new_dummy_data.append(curr)
    

In [10]:
# new_dummy_data
x = []
y = []

In [12]:
for i in range(0, len(new_dummy_data)):
    if i + ORDER_SIZE < len(new_dummy_data):
        x.append(new_dummy_data[i : i+ORDER_SIZE])
        y.append(new_dummy_data[i+ORDER_SIZE][1])
        #print(f"Inputs -> {new_dummy_data[i : i+ORDER_SIZE]} ::::::: Output -> {new_dummy_data[i+ORDER_SIZE][1]} \n\n")

In [13]:
x_train= np.array(x)
y_train = np.array(y)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Reshape,LSTM,Dropout,Activation,Bidirectional

In [15]:
model = Sequential()
model.add(Dense(256, input_shape=(3,3,3),activation = 'relu'))
model.add(Reshape((3,256*3)))
model.add(LSTM(10,return_sequences=True,activation='relu'))
model.add(LSTM(10,activation='relu'))
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(Dropout(0.2))
model.add(Dense(3,activation='softmax'))


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3, 3, 256)         1024      
_________________________________________________________________
reshape (Reshape)            (None, 3, 768)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 3, 10)             31160     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                704       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2

In [15]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [17]:
model.fit(x_train,y_train,epochs=70,batch_size=50,validation_split=0.3)

Epoch 1/70
115/115 [==============================] - 1s 7ms/step - loss: 0.1060 - acc: 0.9450 - val_loss: 0.1639 - val_acc: 0.9464
Epoch 2/70
115/115 [==============================] - 1s 7ms/step - loss: 0.1059 - acc: 0.9445 - val_loss: 0.1316 - val_acc: 0.9469
Epoch 3/70
115/115 [==============================] - 1s 7ms/step - loss: 0.1057 - acc: 0.9441 - val_loss: 0.1404 - val_acc: 0.9440
Epoch 4/70
115/115 [==============================] - 1s 7ms/step - loss: 0.1052 - acc: 0.9474 - val_loss: 0.1308 - val_acc: 0.9444
Epoch 5/70
115/115 [==============================] - 1s 7ms/step - loss: 0.1071 - acc: 0.9441 - val_loss: 0.1255 - val_acc: 0.9473
Epoch 6/70
115/115 [==============================] - 1s 7ms/step - loss: 0.1045 - acc: 0.9464 - val_loss: 0.1399 - val_acc: 0.9501
Epoch 7/70
115/115 [==============================] - 1s 10ms/step - loss: 0.1041 - acc: 0.9462 - val_loss: 0.1271 - val_acc: 0.9452
Epoch 8/70
115/115 [==============================] - 1s 7ms/step - loss: 0

In [85]:
model.output_shape

(None, 3)

In [111]:
model = keras.Sequential()
model.add(keras.layers.Dense(64, input_shape=(3,3,3)))
model.add(keras.layers.Reshape((3, 64 * 3)))
model.add(tf.keras.layers.LSTM(4, return_sequences=True))
model.add(tf.keras.layers.LSTM(4))
model.add(tf.keras.layers.Dense(3))
model.add(tf.keras.layers.Dropout(0.2))
model.add(keras.layers.Activation('softmax'))

In [235]:
model.save("model.h5")

In [16]:
model = Sequential()
model.add(Dense(256, input_shape=(3,3,3),activation = 'relu'))
model.add(Reshape((3,256*3)))
model.add(Bidirectional(LSTM(50,return_sequences=True,activation='relu')))
model.add(Bidirectional(LSTM(50,activation='relu')))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3,activation='sigmoid'))


In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 3, 3, 256)         1024      
_________________________________________________________________
reshape_1 (Reshape)          (None, 3, 768)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 3, 100)            327600    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               60400     
_________________________________________________________________
dense_5 (Dense)              (None, 64)                6464      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)               

In [18]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [19]:
model.fit([x_train, x_train],[y_train, y_train],epochs=70,batch_size=50,validation_split=0.2)

Epoch 1/70
261/261 [==============================] - 6s 10ms/step - loss: 0.5320 - acc: 0.7412 - val_loss: 0.0694 - val_acc: 0.9620
Epoch 2/70
261/261 [==============================] - 2s 7ms/step - loss: 0.1268 - acc: 0.9299 - val_loss: 0.0690 - val_acc: 0.9647
Epoch 3/70
261/261 [==============================] - 2s 8ms/step - loss: 0.1254 - acc: 0.9320 - val_loss: 0.0684 - val_acc: 0.9644
Epoch 4/70
261/261 [==============================] - 2s 8ms/step - loss: 0.1214 - acc: 0.9338 - val_loss: 0.0684 - val_acc: 0.9647
Epoch 5/70
261/261 [==============================] - 2s 8ms/step - loss: 0.1198 - acc: 0.9373 - val_loss: 0.0681 - val_acc: 0.9647
Epoch 6/70
261/261 [==============================] - 2s 8ms/step - loss: 0.1243 - acc: 0.9342 - val_loss: 0.0684 - val_acc: 0.9647
Epoch 7/70
261/261 [==============================] - 2s 8ms/step - loss: 0.1209 - acc: 0.9358 - val_loss: 0.0684 - val_acc: 0.9647
Epoch 8/70
261/261 [==============================] - 2s 8ms/step - loss: 0

KeyboardInterrupt: 

In [65]:
inp_one = keras.layers.Input((3,3,3), name="Input_One")
inp_two = keras.layers.Input((3,3,3), name="Input_Two")

input_one = Dense(256, activation = 'relu', name="Dense_One")(inp_one)
input_two = Dense(128, activation = 'relu')(inp_two)

reshape_one = Reshape((3,256*3))(input_one)
reshape_two = Reshape((3,128*3))(input_two)

bi_lstm = Bidirectional(LSTM(50,return_sequences=True,activation='relu'))(reshape_two)

lstm = LSTM(50,return_sequences=True,activation='relu')(reshape_one)


out = keras.layers.Concatenate()([bi_lstm, lstm])

flatten = keras.layers.Flatten()(out)

final = Dense(3,activation='sigmoid')(flatten)

model = keras.Model(inputs=[inp_one, inp_two], outputs= final)

In [71]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Two (InputLayer)          [(None, 3, 3, 3)]    0                                            
__________________________________________________________________________________________________
Input_One (InputLayer)          [(None, 3, 3, 3)]    0                                            
__________________________________________________________________________________________________
dense_63 (Dense)                (None, 3, 3, 128)    512         Input_Two[0][0]                  
__________________________________________________________________________________________________
Dense_One (Dense)               (None, 3, 3, 256)    1024        Input_One[0][0]                  
___________________________________________________________________________________________

In [74]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=20, monitor="val_acc"),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [75]:
model.fit([x_train, x_train],y_train,epochs=100,batch_size=10,validation_split=0.2, callbacks=callbacks)

Epoch 1/100
1305/1305 [==============================] - 6s 4ms/step - loss: 0.1145 - acc: 0.9414 - val_loss: 0.0653 - val_acc: 0.9654
Epoch 2/100
1305/1305 [==============================] - 5s 4ms/step - loss: 0.1111 - acc: 0.9443 - val_loss: 0.0628 - val_acc: 0.9669
Epoch 3/100
1305/1305 [==============================] - 5s 4ms/step - loss: 0.1069 - acc: 0.9479 - val_loss: 0.0575 - val_acc: 0.9709
Epoch 4/100
1305/1305 [==============================] - 5s 4ms/step - loss: 0.1038 - acc: 0.9470 - val_loss: 0.0545 - val_acc: 0.9724
Epoch 5/100
1305/1305 [==============================] - 6s 4ms/step - loss: 0.1003 - acc: 0.9499 - val_loss: 0.0527 - val_acc: 0.9736
Epoch 6/100
1305/1305 [==============================] - 6s 5ms/step - loss: 0.0944 - acc: 0.9541 - val_loss: 0.0494 - val_acc: 0.9761
Epoch 7/100
1305/1305 [==============================] - 7s 5ms/step - loss: 0.0920 - acc: 0.9548 - val_loss: 0.0490 - val_acc: 0.9782
Epoch 8/100
1305/1305 [==============================] 

In [83]:
test_inp = x_train[0].reshape(-1,3,3,3)

In [91]:
pred = model_two.predict([test_inp, test_inp])

In [92]:
np.round(pred)

array([[0., 0., 1.]], dtype=float32)

In [90]:
model_two = keras.models.load_model('model.h5')
